In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import math

In [ ]:
# cam = cv2.VideoCapture(0)
# face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
# screen_w, screen_h = pyautogui.size()





# Initialize camera and face mesh
cam = cv2.VideoCapture(0)
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
screen_w, screen_h = pyautogui.size()

# For smooth cursor movement
pyautogui.PAUSE = 0
pyautogui.FAILSAFE = False
smoothing_factor = 0.5
prev_x, prev_y = 0, 0

# Blink detection variables
blink_counter = 0
blink_threshold = 3
click_performed = False

# Calibration variables
calibration_points = [(0.25, 0.25), (0.75, 0.25), (0.5, 0.5), (0.25, 0.75), (0.75, 0.75)]
calibration_index = 0
calibrating = False
calibration_data = []

In [ ]:
# while True:
#     _, frame = cam.read()
#     frame = cv2.flip(frame, 1)
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     output = face_mesh.process(rgb_frame)
#     landmark_points = output.multi_face_landmarks
#     frame_h, frame_w, _ = frame.shape
#     if landmark_points:
#         landmarks = landmark_points[0].landmark
#         for id, landmark in enumerate(landmarks[474:478]):
#             x = int(landmark.x * frame_w)
#             y = int(landmark.y * frame_h)
#             cv2.circle(frame, (x, y), 3, (0, 255, 0))
#             if id == 1:
#                 screen_x = screen_w * landmark.x
#                 screen_y = screen_h * landmark.y
#                 pyautogui.moveTo(screen_x, screen_y)
#         left = [landmarks[145], landmarks[159]]
#         for landmark in left:
#             x = int(landmark.x * frame_w)
#             y = int(landmark.y * frame_h)
#             cv2.circle(frame, (x, y), 3, (0, 255, 255))
#         if (left[0].y - left[1].y) < 0.004:
#             pyautogui.click()
#             pyautogui.sleep(1)
#     cv2.imshow('Eye Controlled Mouse', frame)
#     cv2.waitKey(1)







while True:
    _, frame = cam.read()
    if not _:
        break
        
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape
    
    if landmark_points:
        landmarks = landmark_points[0].landmark
        
        # Eye landmarks for cursor control (right eye)
        eye_points = landmarks[474:478]
        for id, landmark in enumerate(eye_points):
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 0))
            
            if id == 1:  # Use a specific point for cursor control
                # Smooth the cursor movement
                screen_x = screen_w * landmark.x
                screen_y = screen_h * landmark.y
                
                if calibrating:
                    # During calibration, just show the point
                    cv2.putText(frame, "Calibrating... Look at the dot", (50, 50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    # Apply smoothing
                    smooth_x = prev_x + (screen_x - prev_x) * smoothing_factor
                    smooth_y = prev_y + (screen_y - prev_y) * smoothing_factor
                    pyautogui.moveTo(smooth_x, smooth_y)
                    prev_x, prev_y = smooth_x, smooth_y
        
        # Blink detection (left eye)
        left = [landmarks[145], landmarks[159]]  # top and bottom of left eye
        for landmark in left:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))
        
        # Calculate eye opening distance
        vertical_dist = abs(left[0].y - left[1].y)
        
        # Blink detection
        if vertical_dist < 0.01:  # Increased threshold for more reliable detection
            blink_counter += 1
            if blink_counter >= blink_threshold and not click_performed:
                pyautogui.click()
                click_performed = True
                cv2.putText(frame, "CLICK", (50, 100), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            blink_counter = 0
            click_performed = False
        
        # Draw eye opening distance
        cv2.putText(frame, f"Eye Open: {vertical_dist:.4f}", (50, 150), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
    
    # Calibration mode
    if calibrating:
        target_x = int(calibration_points[calibration_index][0] * frame_w)
        target_y = int(calibration_points[calibration_index][1] * frame_h)
        cv2.circle(frame, (target_x, target_y), 10, (0, 0, 255), -1)
        
        # Check if user has looked at the point long enough
        # (In a real implementation, you'd collect data points here)
    
    # Display instructions
    cv2.putText(frame, "Press 'c' to calibrate, 'q' to quit", (10, 20), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    cv2.imshow('Eye Controlled Mouse', frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('c'):
        calibrating = not calibrating
        if calibrating:
            calibration_index = 0
            calibration_data = []
            print("Starting calibration...")

cam.release()
cv2.destroyAllWindows()